# Notebook para acesso ao Receita Data - dados de viajantes

In [17]:
import getpass
import pandas as pd
from datalake_serpro import handler_dl

In [18]:
user = getpass.getuser() + '@DATALAKE.SERPRO'
senha = getpass.getpass(f'Senha para kinit {user}:')

Senha para kinit 25052288840@DATALAKE.SERPRO:········


In [19]:
commands = f'echo "{senha}" | /opt/anaconda3/bin/kinit {user}'
! {commands}

Password for 25052288840@DATALAKE.SERPRO: 


In [20]:
! /opt/anaconda3/bin/klist

Ticket cache: FILE:/tmp/krb5cc_1227403730
Default principal: 25052288840@DATALAKE.SERPRO

Valid starting       Expires              Service principal
07/02/2021 08:07:36  07/02/2021 18:07:36  krbtgt/DATALAKE.SERPRO@DATALAKE.SERPRO
	renew until 07/09/2021 08:07:36


# Rodar

In [21]:
def get_viagens_cpfs(lista_cpfs):
    sql_vu = 'SELECT DISTINCT cpf_vu as cpf, codigo_vu FROM coana.edbv_viajante_unico_geral WHERE cpf_vu in ("%s")'
    sql_viagens = 'SELECT * FROM coana.edbv_dados_voos WHERE codigo_vu in (%s)'
    df_vu = pd.read_sql(sql_vu % '", "'.join(lista_cpfs), con)
    lista_vus = ['%d' % codigo_vu for codigo_vu in df_vu.codigo_vu.values]
    df_viagens = pd.read_sql(sql_viagens % ', '.join(lista_vus), con)
    df_final = df_vu.merge(df_viagens, right_on='codigo_vu', left_on='codigo_vu')
    return df_final

In [22]:
def get_nomes_cnpjs(lista_cpfs):
    sql_cnpjs ='SELECT DISTINCT b_cd_cnpj_emph as cnpj, nm_cnpj_emph_empresarial as nome ' + \
               'FROM cnpj.wd_cnpj_esth where b_cd_cnpj_emph in (%s)'
    lista_cnpjs = []
    for row in lista_cpfs:
        try:
            item = str(int(row[:8]))
            lista_cnpjs.append(item)
        except:
            pass
    df_cnpjs = pd.read_sql(sql_cnpjs % ', '.join(lista_cnpjs), con)
    return df_cnpjs

In [23]:
def get_nomes_cpfs(lista_cpfs):
    sql_cpfs = 'SELECT DISTINCT b_cd_cnpf_cpf as cpf, nm_cnpf_cpfa_nome as nome ' + \
               ' FROM cnpf.wd_cnpf_cpfa where b_cd_cnpf_cpf in (%s)'
    lista = []
    for row in lista_cpfs:
        try:
            item = str(int(row))
            lista.append(item)
        except:
            pass
    df_cpfs = pd.read_sql(sql_cpfs % ', '.join(lista), con)
    return df_cpfs

In [24]:
def get_dsis_cpfs(lista_cpfs):
    sql_dsis = '''
    SELECT nm_dsi as numero,
    nr_impdr_expdr as consignatario,
    dt_dia_regis as data_registro, 
    nr_rep_legal as despachante,
    max(nx_esp_produto_dsi) as descricao
    FROM coana.importacao_dsi WHERE nr_impdr_expdr in ("%s")
    GROUP BY nm_dsi, nr_impdr_expdr, dt_dia_regis, nr_rep_legal
    '''
    lista = []
    for row in lista_cpfs:
        try:
            item = str(int(row))
            lista.append(item)
        except:
            pass
    df_cpfs = pd.read_sql(sql_dsis % '", "'.join(lista), con)
    return df_cpfs

In [25]:
cpfs = pd.read_csv('viagem.csv', header=None, names=['cpf'])
cpfs2 = pd.read_csv('pessoa.csv', header=None, names=['cpf'])
cpfs3 = pd.read_csv('empresa.csv', header=None, names=['cpf'])
cpfs_dsi = pd.read_csv('dsi.csv', header=None, names=['cpf'])

In [26]:
with handler_dl(DEBUG=True, timeout=300) as con:
    df_viagens = get_viagens_cpfs(cpfs.cpf.values)


Conectado ao Servidor impalaharfb.srv.datalake.serpro.


In [27]:
df_viagens.to_csv('viagens.csv')

In [28]:
with handler_dl(DEBUG=True, timeout=300) as con:
    df_cnpjs = get_nomes_cnpjs(cpfs3.cpf.values)

df_cnpjs.to_csv('cnpjs.csv')


Conectado ao Servidor impalaharfb.srv.datalake.serpro.


In [29]:
with handler_dl(DEBUG=True, timeout=300) as con:
    df_cpfs = get_nomes_cpfs(cpfs2.cpf.values)

df_cpfs.to_csv('cpfs.csv')


Conectado ao Servidor impalaharfb.srv.datalake.serpro.


In [30]:
with handler_dl(DEBUG=True, timeout=300) as con:
    df_dsis = get_dsis_cpfs(cpfs_dsi.cpf.values)

df_dsis.to_csv('dsis.csv')

Conectado ao Servidor impalaharfb.srv.datalake.serpro.
